In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [7]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'A curious fact about Cristiano Ronaldo is that he is the only player in history to have won league titles in England (Premier League with Manchester United), Spain (La Liga with Real Madrid), and Italy (Serie A with Juventus). This showcases his incredible adaptability and success across different top European football leagues.'

# Use of .bind() to add arguments to a Runnable in a LCEL Chain
- For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [8]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

'A curious fact about Cristiano '

# Combining LCEL Chains

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [12]:
chain.invoke("Chamberlain")

'Neville Chamberlain was the British Prime Minister known for his policy of appeasement toward Adolf Hitler prior to World War II.'

# Combined chain

In [13]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [14]:
composed_chain.invoke({"politician": "Lincoln"})

'Yes, Abraham Lincoln is widely regarded as having had a positive impact on humanity. As the 16th President of the United States, he led the nation through its most challenging period, the Civil War, striving to preserve the Union. Importantly, he played a crucial role in ending slavery in the United States through the issuance of the Emancipation Proclamation in 1863 and by advocating for the passage of the 13th Amendment, which abolished slavery. His leadership helped to advance human rights and set a foundation for greater equality, making his legacy largely positive for humanity.'

# A chain inside another chain

In [16]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "Vietnamese"})

'Châu Âu'

# LCEL chain at work in a typical RAG app

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

In [ ]:
import bs4
import os
import sys
import warnings
from contextlib import redirect_stderr
from io import StringIO
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",), # extract all infomation from a website
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header") # crawl only from these class
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

# Use hub.pull() - still works but shows deprecation warning
# For newer version, use: from langsmith import pull_prompt
prompt = hub.pull("rlm/rag-prompt") # use the prompt from prompt hub instead of writing your own prompt

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?") # A few small error for the older version but still workable
# updating the requirements.txt will fix it

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
c:\Users\admin\anaconda3\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
c:\Users\admin\anaconda3\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


'Task Decomposition is the process of breaking down a complex task into smaller, manageable steps or subgoals to facilitate planning and problem-solving. Techniques like Chain of Thought (CoT) prompt models to "think step by step," while methods like Tree of Thoughts explore multiple reasoning paths. It can be performed by LLMs with simple prompts, task-specific instructions, human inputs, or by using external classical planners via intermediate representations like PDDL.'

In [ ]:
rag_chain.invoke("What is Task Decomposition?")